<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/retrievers/pathway_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pathway Retriever

In [ ]:
from llama_index.retrievers import PathwayRetriever, PathwayVectorServer
import pathway as pw

In [ ]:
import getpass
import os
import pathway as pw

# omit if embedder of choice is not OpenAI
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Define data sources for Pathway

In [ ]:
data_sources = []
data_sources.append(
    pw.io.fs.read(
        "../data/paul_graham",
        format="binary",
        mode="streaming",
        with_metadata=True,
    )  # This creates a `pathway` connector that tracks
    # all the files in the sample_documents directory
)

## Define Transformation pipeline

In [ ]:
from llama_index.embeddings import OpenAIEmbedding
from llama_index.node_parser import TokenTextSplitter

embed_model = OpenAIEmbedding(embed_batch_size=10)

transformations_example = [
    TokenTextSplitter(
        chunk_size=150,
        chunk_overlap=10,
        separator=" ",
    ),
    embed_model,
]

## Run the Server

In [ ]:
pr = PathwayVectorServer(
    *data_sources,
    transformations=transformations_example,
)

# Define the Host and port that Pathway will be on
PATHWAY_HOST = "127.0.0.1"
PATHWAY_PORT = 8754

pr.run_server(
    host=PATHWAY_HOST, port=PATHWAY_PORT, with_cache=False, threaded=True
)

## Create Retriever for llama-index

In [ ]:
pr = PathwayRetriever(host=PATHWAY_HOST, port=PATHWAY_PORT)

In [ ]:
pr.retrieve(str_or_query_bundle="something")